# Faster R-CNN all-in-one 

In [5]:
# !apt-get install protobuf-compiler python-pil python-lxml python-tk
# !git clone https://github.com/tensorflow/models.git
# !protoc object_detection/protos/*.proto --python_out=.

# !pip install jupyter_innotater
# !apt-get install unzip

import os
import sys
import itertools
import random
import datetime

import numpy as np
import cv2
import pandas as pd
import json

%load_ext tensorboard
import tensorflow as tf
sys.path.append("./models/research")
from object_detection.utils import label_map_util

import generate_tfrecord as gt

# make some folders to organize our files

# ! mkdir pre-trained-model
# %cd ./pre-trained-model
# !wget --no-check-certificate https://ndownloader.figshare.com/files/26260501
# %cd ..
! mkdir training
! mkdir exported-model
! mkdir annotations

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


A subdirectory or file training already exists.
A subdirectory or file exported-model already exists.
A subdirectory or file annotations already exists.


## Annotating frames from a movie
If you wish to annotate frames from a movie, first convert them into images and save them in a new folder called 'images' in the same directory as this. **If your data is already saved as images, skip this step and don't run the following cell.**
You should specify the filepath of your movie you want to annotate as the 

```
video_path 
```

variable below

In [ ]:

! mkdir images
video_path = "worm_pump02.mp4"
vid = cv2.VideoCapture(video_path)
i = 1
while vid.isOpened():
    ret, image_np = vid.read()
    if ret:
        frame_name = 'img' + str(i) + '.jpg'
        cv2.imwrite(os.path.join("images", frame_name), image_np)
    else:
        break
    print("Processing frame no %s" % i)
    i += 1
vid.release()

## Annotating images
Once your data is in image form, we will use Innotator to annotate it. If you did not run the code above to convert a movie to a set of images, make a folder called "images" in the same folder as this and upload your images there. **Make sure the images are named as imgxx.jpg, where xx is a number**

Depending on your needs, you may not need to annotate all that many images. Here we only demonstrate fine-tuning a pre-existing model, and while we used between ~100 - 1000 annotations for our models, if you want very high accuracy, you may need to annotate more images. 

Once you run the code cell below, you will see an interface for annotation appear. Draw boxes around the types of objects you wish to annotate ('classes'). By default, the classes we use below are those in our paper, 'worm' and 'egg'. Advance to the next image by clicking ‘Next’ or pressing ‘n’ on the keyboard (provided the annotation tool has focus).

In [2]:
from jupyter_innotater import *
import glob

img_path = './images'
worm_image_files = glob.glob('./images/*.JPG')
worm_image_files.sort(key=os.path.getmtime)

# 'Repeats' is the maximum number of objects you expect to annotate in each image
repeats = 7
# Feel free to modify the classes, or types of objects, you want to identify
classes = ['worm', 'egg']

# Binary flag to indicate an image should be excluded from dataset
targets_exclude = np.zeros((len(worm_image_files), 1), dtype='int') 

# set up arrays to load annotation info into
targets_classes = np.zeros((len(worm_image_files), len(classes)*repeats), dtype='int')
targets_bboxes = np.zeros((len(worm_image_files), len(classes)*repeats, 4), dtype='int') # (xmin,ymin,w,h) for each animal

Innotater(
    [
        ImageInnotation(worm_image_files, path='./images'), # Display the image itself
        TextInnotation(worm_image_files, multiline=False) # Display the image filename
    ],
    [
        BinaryClassInnotation(targets_exclude, name='Exclude'), # Checkbox
        RepeatInnotation(
            (BoundingBoxInnotation, targets_bboxes), # Individual animal bounding box
            (MultiClassInnotation, targets_classes,
                {'name':'object', 'classes':classes, 'dropdown':True}), # Per-annotation dropdown
            max_repeats=len(classes)*repeats, min_repeats=1
        )
    ]
)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00…

## Convert annotations to tfrecord format
Tensorflow uses the tfrecord format as input to our model. To convert to this format, first we'll prepare the annotations, save them as a csv file, and build a label map from our list of classes. We will split up annotations now so that the precision and accuracy of the model can be tested later. We choose images at random and choose a 90/ 10 train/test split for the data.

In [7]:
# reshape data so that the structure is flat
flat_bboxes = np.reshape(targets_bboxes, (-1, 4))
flat_classes = np.reshape(targets_classes, (-1,))
flat_classes = [classes[x] for x in flat_classes]

# Grab widths and heights for each image 
widths = np.zeros((len(worm_image_files),), dtype='int')
heights = np.zeros((len(worm_image_files),), dtype='int')
for i, im_file in enumerate(worm_image_files):
    im = cv2.imread(os.path.join('./images', im_file))
    heights[i], widths[i], d = im.shape
            
# it can be helpful to have annotations stored in a generic csv as well, so we'll prepare annotations and do that here.
data = {'filename': np.repeat(worm_image_files, len(classes)*repeats), 'xmin': flat_bboxes[:,0], 'ymin': flat_bboxes[:,1], 
        'xmax': np.add(flat_bboxes[:,0], flat_bboxes[:,2]), 'ymax': np.add(flat_bboxes[:,1], flat_bboxes[:,3]), 
        'width': np.repeat(widths, len(classes)*repeats), 'height':np.repeat(heights, len(classes)*repeats),
        'exclude': np.repeat(np.reshape(targets_exclude, (-1,)), len(classes)*repeats), 'class': flat_classes}
df = pd.DataFrame(data)
# screen out any un-annotated frames or any images marked by the annotator as 'Exclude'
df_annotated = df[(df['exclude'] == 0) & (df['xmin'] != df['xmax'])]

# if you want to change the train/ test split, you can decrease number of training images (which will increase the testing images) by decreasing this. It must be between 0 and 1
train_split = 0.9 
annotated_images = df_annotated['filename'].unique()
num_train_images = int(train_split*len(annotated_images))
train_images = random.sample(list(annotated_images), num_train_images)
test_images = list(set(annotated_images).difference(set(train_images)))
test_idx = df_annotated['filename'].isin(test_images)
test_or_train = ['test' if test_i else 'train' for test_i in test_idx]
df_annotated.insert(9, 'test_or_train', test_or_train)

# save csv to annotations directory
csv_filepath = './annotations/bounding_boxes.csv'
df_annotated.to_csv(csv_filepath, index=False)

# next, make a label file so we know how to map the names of classes to a number. 'worm' will map to 1, 'egg' will map to 2, and so on
label_map_path = './training/label_map.pbtxt'
gt.write_label_map(classes, label_map_path)

Now build tfrecord files for train and test data, saving them in the 'annotations' folder.


In [8]:
def write_tf_record(annotations, tfrecord_path, img_path, label_map):
    tf_writer = tf.io.TFRecordWriter(tfrecord_path)
    for annotation in annotations:
        tf_example = gt.create_tf_example(annotation, img_path, label_map)
        tf_writer.write(tf_example.SerializeToString())

    tf_writer.close()
    output_path = os.path.join(os.getcwd(), tfrecord_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


# and now we'll convert annotations to a tfrecord
label_map = label_map_util.get_label_map_dict(label_map_path)
all_annotations = pd.read_csv(csv_filepath)
img_path = './images'

# tfrecord for train annotations
tfrecord_train_path = './annotations/train.record'
train_annotations = all_annotations[all_annotations['test_or_train'].isin(['train'])]
grouped_train_annotations = gt.split(train_annotations, 'filename')
write_tf_record(grouped_train_annotations, tfrecord_train_path, img_path, label_map)

# tfrecord for test annotations
tfrecord_test_path = './annotations/test.record'
test_annotations = all_annotations[all_annotations['test_or_train'].isin(['test'])]
grouped_test_annotations = gt.split(test_annotations, 'filename')
write_tf_record(grouped_test_annotations, tfrecord_test_path, img_path, label_map)

Successfully created the TFRecords: C:\Users\kebel\Dropbox (GaTech)\rcnn_extras\code dump\./annotations/train.record
Successfully created the TFRecords: C:\Users\kebel\Dropbox (GaTech)\rcnn_extras\code dump\./annotations/test.record


## Train

In [ ]:
def makeWindowsCmdPath(path):
    return '\"' + str(path) + '\"'

log_dir = r"C:\Users\kebel\Dropbox%20(GaTech)\rcnn_extras\code%20dump\training\train"
%tensorboard --logdir {makeWindowsCmdPath(log_dir)}

!python model_main_tf2.py --model_dir=training --pipeline_config_path=training/faster_rcnn.config --alsologtostderr 

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 27420.

## Save model

In [ ]:
!python object_detection/export_inference_graph.py \
--input_type=image_tensor \
--pipeline_config_path=training/faster_rcnn.config \
--trained_checkpoint_prefix=/training/model.ckpt-1691 \
--output_directory=exported-model

## Test model
First, take a look at how well the model performs on our test data we decided on above.

In [4]:
from IPython.display import Video, Image, display
import glob
from inference_code import inferencing_tools

! mkdir inferencing

path_to_frozen_graph = './exported-model/frozen_inference_graph.pb'
# we've already defined this above, but just in case
path_to_labels = './training/label_map.pbtxt'
save_to_hdf = True
save_path = './inferencing'
h5_file = os.path.join(save_path, 'test_detections.h5')

# every inference you make with this 'cnn' object will be saved in the same h5 file
cnn = inferencing_tools.CNN(path_to_frozen_graph, path_to_labels, save_to_hdf, h5_file)

# take a look at qualitatively how well the model performs on our test images
test_image_paths = test_images

for path in test_image_paths:
    image_np = cv2.imread(path)
    #inferencing happens in this call
    worm_boxes, egg_boxes = cnn.get_eggs_and_worms(image_np, path)

# Now that we have all the detections, label them on the test data and visualize the detections on each test image.
inferencing_tools.label_all_detections_from_h5(h5_file, test_image_paths, save_path)
for image_name in glob.glob('./inferencing/*.JPG'): #assuming JPG
    display(Image(filename=image_name))


To inference on other data, we will use similar methods to that above. Here's an example that detects all worms and eggs from .jpg images in a folder, saves them to an h5 file, and then overlays the detections on top of the original images.

In [ ]:
path_to_frozen_graph = './exported-model/frozen_inference_graph.pb'
# we've already defined this above, but just in case
path_to_labels = './training/label_map.pbtxt'
save_to_hdf = True
save_path = './inferencing'
h5_file = os.path.join(save_path, 'folder_detections.h5')

# every inference you make with this 'cnn' object will be saved in the same h5 file
cnn = inferencing_tools.CNN(path_to_frozen_graph, path_to_labels, save_to_hdf, h5_file)

# take a look at qualitatively how well the model performs on our test images
image_dir = './images'

image_paths = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))
                             and f.endswith('.jpg')]

for path in image_paths:
    image_np = cv2.imread(path)
    #inferencing happens in this call - you can directly use the boxes this returns if you wish. They are also saved in the h5 file
    worm_boxes, egg_boxes = cnn.get_eggs_and_worms(image_np, path)

# Now that we have all the detections, label them on the test data and visualize the detections on each test image.
inferencing_tools.label_all_detections_from_h5(h5_file, image_paths, save_path)
for image_name in glob.glob('./inferencing/*.JPG'): #assuming JPG
    display(Image(filename=image_name))

Here's an example that detects both eggs and worms for each frame in a video, saves the detections to an h5 file, and then overlays just the detection of the top-scoring worm on top of the original images.

In [ ]:
path_to_frozen_graph = './exported-model/frozen_inference_graph.pb'
# we've already defined this above, but just in case
path_to_labels = './training/label_map.pbtxt'
save_to_hdf = True
save_path = './inferencing'
h5_file = os.path.join(save_path, 'movie_detections.h5')

# every inference you make with this 'cnn' object will be saved in the same h5 file
cnn = inferencing_tools.CNN(path_to_frozen_graph, path_to_labels, save_to_hdf, h5_file)

# detect and visualize detections from movie
movie_path = ''
save_file = ''

vid = cv2.VideoCapture(source_data)
idx = 1
while vid.isOpened():
    ret, image = vid.read()
    if ret:
        #inferencing happens in this call
        worm_boxes, egg_boxes = cnn.get_eggs_and_worms(image, idx)
    else:
        break
    print("Processing frame no %s" % i)
    idx += 1
vid.release()

inferencing_tools.label_all_detections_from_h5(h5_file, movie_path, save_file)

# if the input of the inferencing is a video, the output will be a video, otherwise it will be an image
Video(save_file)